In [0]:
!pip install -U -q PyDrive

In [0]:
!pip install hyperdash
from hyperdash import monitor_cell
!hyperdash signup --github

  Created wheel for hyperdash: filename=hyperdash-0.15.3-cp36-none-any.whl size=28553 sha256=9ad04ccc7dedf8e0be224f17df072f9081c50a7d51a8f27c2b4f4852d693bc5a
  Stored in directory: /root/.cache/pip/wheels/62/5f/af/bbcaeb6570e4904c14fb4c1b70fee559a3788182ce4d104ce7
Successfully built hyperdash
Opening browser, please wait. If something goes wrong, press CTRL+C to cancel.
 SSH'd into a remote machine, or just don't have access to a browser? Open this link in any browser and then copy/paste the provided access token: https://hyperdash.io/oauth/github/start?state=client_cli_manual 
Waiting for Github OAuth to complete.
If something goes wrong, press CTRL+C to cancel.
Access token: nCTFZUSf3c/WdqZzokhzr5WyWnmIDWEFt6iALgdoNLs=
Successfully logged in! We also installed: yWgexKOerZf5n/R8HHG19FhvDS6ZndyExaBZ68fhmxM= as your default API key


In [0]:
from tensorflow.keras.callbacks import Callback
from hyperdash import Experiment

class Hyperdash(Callback):
    def __init__(self, entries, exp):
        super(Hyperdash, self).__init__()
        self.entries = entries
        self.exp = exp

    def on_epoch_end(self, epoch, logs=None):
        for entry in self.entries:
            log = logs.get(entry)            
            if log is not None:
                self.exp.metric(entry, log)

In [0]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!ls ./drive/My\ Drive/

 01.model
 amidaTest2.py
 bert-master.zip
 bert_x.npy
 bow_train_x.npz
 cabocha-0.69.tar.bz2
'Colab Notebooks'
 C言語課題.gsite
 doc2vec-dmpv_x.npy
 doc2vec.py
 doc2vec_x.npy
 export
 fasttext-IMDB.bin
 fasttext-neologd.bin
 fasttext-normal.bin
 fasttext_x_hier.npy
 fasttext_x_max.npy
 fasttext_x_mean.npy
 Japanese_L-12_H-768_A-12_E-30_BPE_transformers.zip
 ja.zip
 LBa.csv
 LBa.gsheet
 modified.pdf
 n-gram-tf-idf_x.npz
 n-gram_x.npz
 nmf.pdf
 Saitaihou.pdf
 sdv.npy
 tf-idf_x.npz
 train-val_pre.tsv
 train-val-pre.tsv
 train-val-small.tsv
 train-val-wakati-juman2.tsv
 train-val-wakati-juman-nva.tsv
 train-val-wakati-juman.tsv
 train-val-wakati-nva.tsv
 train-val-wakati.tsv
 vector_data
 word2vec_x_mean.npy
 y_full.npy
 六義園.gdoc
 無題のプレゼンテーション.gslides
 試作


In [0]:
%cp ./drive/My\ Drive/sdv.npy ./
%cp ./drive/My\ Drive/y_full.npy ./

In [0]:
 %mkdir model
%mkdir model/tuning

In [0]:
 import os,sys
sys.path.append('../')

import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from typing import Callable, List, Optional, Tuple, Union
from sklearn.model_selection import learning_curve
from scipy import sparse
from scipy.sparse import load_npz
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

import datetime
import logging

from sklearn.externals import joblib
from scipy.sparse import load_npz
from abc import ABCMeta, abstractmethod

from tqdm import tqdm
from keras.callbacks import EarlyStopping
from keras.layers import SpatialDropout1D, Bidirectional, GlobalMaxPool1D
from keras.layers.recurrent import LSTM
from keras.layers.advanced_activations import ReLU, PReLU
from keras.layers.core import Dense, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score

from gensim.models import KeyedVectors

class Util:
    @classmethod
    def dump(cls, value, path):
        os.makedirs(os.path.dirname(path), exist_ok=True)
        joblib.dump(value, path, compress=True)

    @classmethod
    def load(cls, path):
        return joblib.load(path)

class Logger:
    def __init__(self):
        self.general_logger = logging.getLogger('general')
        self.result_logger = logging.getLogger('result')
        stream_handler = logging.StreamHandler()
        file_general_handler = logging.FileHandler('./model/general.log')
        file_result_handler = logging.FileHandler('./model/result.log')
        if len(self.general_logger.handlers) == 0:
            self.general_logger.addHandler(stream_handler)
            self.general_logger.addHandler(file_general_handler)
            self.general_logger.setLevel(logging.INFO)
            self.result_logger.addHandler(stream_handler)
            self.result_logger.addHandler(file_result_handler)
            self.result_logger.setLevel(logging.INFO)

    def info(self, message):
        # 時刻をつけてコンソールとログに出力
        self.general_logger.info('[{}] - {}'.format(self.now_string(), message))

    def result(self, message):
        self.result_logger.info(message)

    def result_ltsv(self, dic):
        self.result(self.to_ltsv(dic))

    def result_scores(self, run_name, scores):
        # 計算結果をコンソールと計算結果用ログに出力
        dic = dict()
        dic['name'] = run_name
        dic['score'] = np.mean(scores)
        for i, score in enumerate(scores):
            dic[f'score{i}'] = score
        self.result_ltsv(dic)

    def now_string(self):
        return str(datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S'))

    def to_ltsv(self, dic):
        return '\t'.join(['{}:{}'.format(key, value) for key, value in dic.items()])

class Model(metaclass=ABCMeta):
    def __init__(self, run_fold_name: str, params: dict) -> None:
        """コンストラクタ
        :param run_fold_name: ランの名前とfoldの番号を組み合わせた名前
        :param params: ハイパーパラメータ
        """
        self.run_fold_name = run_fold_name
        self.params = params
        self.model = None

    @abstractmethod
    def train(self, tr_x: np.array, tr_y: np.array,
              va_x: Optional[np.array] = None,
              va_y: Optional[np.array] = None) -> None:
        """モデルの学習を行い、学習済のモデルを保存する
        :param tr_x: 学習データの特徴量
        :param tr_y: 学習データの目的変数
        :param va_x: バリデーションデータの特徴量
        :param va_y: バリデーションデータの目的変数
        """
        pass

    @abstractmethod
    def predict(self, te_x: np.array) -> np.array:
        """学習済のモデルでの予測値を返す
        :param te_x: バリデーションデータやテストデータの特徴量
        :return: 予測値
        """
        pass

    @abstractmethod
    def score(self, te_x: np.array, te_y: np.array) -> float:
        """学習済のモデルでのスコア値を返す
        :te_x: np.array
        :te_y: np.array
        :return: 予測値
        """
        pass

    @abstractmethod
    def save_model(self, feature: str) -> None:
        """モデルの保存を行う"""
        pass

    @abstractmethod
    def load_model(self, feature: str) -> None:
        """モデルの読み込みを行う"""
        pass


logger = Logger()

def load_x_train(features, sparse=False):
    if features == "bow":
        matrix =load_npz('./bow_train_x.npz').astype('float64')
    elif features == "n-gram":
        matrix = load_npz('./n-gram_x.npz').astype('float64')
    elif features == "tf-idf":
        matrix = load_npz('./tf-idf_x.npz').astype('float64')
    elif features == "n-gram-tf-idf":
        matrix = load_npz('./n-gram-tf-idf_x.npz').astype('float64')
    elif features == "word2vec_mean":
        matrix = np.load('./word2vec_x_mean.npy', allow_pickle = True)
    elif features == "word2vec_max":
        matrix = np.load('./word2vec_x_max.npy', allow_pickle = True)
    elif features == "word2vec_concat":
        l = np.load('./word2vec_x_mean.npy', allow_pickle = True)
        r = np.load('./word2vec_x_max.npy', allow_pickle = True)
        matrix = np.hstack((l, r))
    elif features == "word2vec_hier":
        matrix = np.load('../vec/word2vec_x_hier.npy', allow_pickle = True)
    elif features == "fasttext_mean":
        matrix = np.load('../vec/fasttext_x_mean.npy', allow_pickle = True)
    elif features == "fasttext_max":
        matrix = np.load('./fasttext_x_max.npy', allow_pickle = True)
    elif features == "fasttext_concat":
        l = np.load('./fasttext_x_mean.npy', allow_pickle = True)
        r = np.load('./fasttext_x_max.npy', allow_pickle = True)
        matrix = np.hstack((l, r))
    elif features == "fasttext_hier":
        matrix = np.load('./fasttext_x_hier.npy', allow_pickle = True)
    elif features == "doc2vec-dbow":
        matrix = np.load('./doc2vec_x.npy', allow_pickle=True)
    elif features == "doc2vec-dmpv":
        matrix = np.load('./doc2vec-dmpv_x.npy', allow_pickle=True)
    elif features == "doc2vec-concat":
        l = np.load('./doc2vec_x.npy', allow_pickle=True)
        r = np.load('./doc2vec-dmpv_x.npy', allow_pickle=True)
        matrix = np.hstack((l, r))
    elif features == "sdv":
        matrix = np.load('./sdv.npy', allow_pickle=True)
    elif features == "bert":
        matrix = np.load('../vec/bert_x.npy', allow_pickle=True)
    elif features == "raw_text":
        df = pd.read_table("../data/train-val_pre.tsv", index_col=0)
        matrix = np.array(df["text"], dtype=str)
    return matrix

def load_y_train(features):
    if features == "bow_nva":
        return np.load('../vec/bow_train_y_nva.npy', allow_pickle = True)
    elif features == "raw_text":
        df = pd.read_table("../data/train-val_pre.tsv", index_col=0)
        return np.array(df["label"], dtype=int)
    else:
        return np.load('./y_full.npy', allow_pickle=True).astype('int')


from scipy.sparse import issparse


# tensorflowの警告抑制
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

class ModelMLP(Model):
    def __init__(self, run_fold_name, **params):
        super().__init__(run_fold_name, params)

    def train(self, tr_x, tr_y, va_x=None, va_y=None):
        # scaling
        validation = va_x is not None
        exp = Experiment("MLP-Twitter")
        hd_callback = Hyperdash(["val_loss", "loss", "val_accuracy", "accuracy"], exp)

        # パラメータ
        nb_classes = 5
        input_dropout = self.params['input_dropout']
        hidden_layers = int(self.params['hidden_layers'])
        hidden_units = int(self.params['hidden_units'])
        hidden_activation = self.params['hidden_activation']
        hidden_dropout = self.params['hidden_dropout']
        batch_norm = self.params['batch_norm']
        optimizer_type = self.params['optimizer']['type']
        optimizer_lr = self.params['optimizer']['lr']
        batch_size = int(self.params['batch_size'])
        nb_epoch = int(self.params['nb_epoch'])

        if issparse(tr_x):
            scaler = StandardScaler(with_mean=False)
        else:
            scaler = StandardScaler()
        scaler.fit(tr_x)
        tr_x = scaler.transform(tr_x)
        tr_y = np_utils.to_categorical(tr_y, num_classes=nb_classes)
        if validation:
            va_x =  scaler.transform(va_x)
            va_y = np_utils.to_categorical(va_y, num_classes=nb_classes)

        self.model = Sequential()
        # input layer
        self.model.add(Dropout(input_dropout, input_shape=(tr_x.shape[1],)))
        # 中間層
        for i in range(hidden_layers):
            self.model.add(Dense(hidden_units))
            if batch_norm == 'before_act':
                self.model.add(BatchNormalization())
            if hidden_activation == 'prelu':
                self.model.add(PReLU())
            elif hidden_activation == 'relu':
                self.model.add(ReLU())
            else:
                raise NotImplementedError
            self.model.add(Dropout(hidden_dropout))

        # 出力層
        self.model.add(Dense(nb_classes, activation='softmax'))

        # オプティマイザ
        if optimizer_type == 'sgd':
            optimizer = SGD(lr=optimizer_lr, decay=1e-6, momentum=0.9, nesterov=True)
        elif optimizer_type == 'adam':
            optimizer = Adam(lr=optimizer_lr, beta_1=0.9, beta_2=0.999, decay=0.)
        else:
            raise NotImplementedError

        # 目的関数、評価指標などの設定
        self.model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

        # エポック数、アーリーストッピング
        # あまりepochを大きくすると、小さい学習率のときに終わらないことがあるので注意
        patience = 10
        # 学習の実行
        if validation:
            early_stopping = EarlyStopping(monitor='val_loss', patience=patience,
                                            verbose=0, restore_best_weights=True)
            history = self.model.fit(tr_x, tr_y, epochs=nb_epoch, batch_size=batch_size, verbose=0,
                                validation_data=(va_x, va_y), callbacks=[early_stopping])
        else:
            history = self.model.fit(tr_x, tr_y, nb_epoch=nb_epoch, batch_size=batch_size, verbose=0)

        self.scaler = scaler
        exp.end()

    def predict(self, te_x):
        te_x = self.scaler.fit_transform(te_x)
        y_pred = self.model.predict(te_x)
        return y_pred

    def score(self, te_x, te_y):
        y_pred = self.predict(te_x)
        return f1_score(np.identity(5)[te_y], np.identity(5)[np.argmax(y_pred, axis=1)], average='samples')

    def save_model(self, feature):
        model_path = os.path.join(f'./model/model/{feature}', f'{self.run_fold_name}.h5')
        scaler_path = os.path.join(f'./model/model/{feature}', f'{self.run_fold_name}-scaler.pkl')
        os.makedirs(os.path.dirname(model_path), exist_ok=True)
        self.model.save(model_path)
        Util.dump(self.scaler, scaler_path)

    def load_model(self, feature):
        model_path = os.path.join(f'./model/model/{feature}', f'{self.run_fold_name}.h5')
        scaler_path = os.path.join(f'./model/model/{feature}', f'{self.run_fold_name}-scaler.pkl')
        self.model = load_model(model_path)
        self.scaler = Util.load(scaler_path)
        
        

/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)
Using TensorFlow backend.


In [0]:
from hyperopt import hp
from hyperopt import fmin, tpe, STATUS_OK, Trials

import matplotlib.pyplot as plt
import seaborn as sns

import gc
gc.collect()

def objective(params):
    global param
    param.update(params)
    model = ModelMLP("MLP", **param)
    model.train(tr_x, tr_y, va_x, va_y)
    va_pred = model.predict(va_x)
    score = log_loss(va_y, va_pred)
    print(f'params: {params}, logloss: {score:.4f}')
    # 情報を記録しておく
    history.append((params, score))
    del model

    return {'loss': score, 'status': STATUS_OK}

if __name__ == '__main__':
    # 基本となるパラメータ
    param = {
        'input_dropout': 0.0,
        'hidden_layers': 3,
        'hidden_units': 96,
        'hidden_activation': 'relu',
        'hidden_dropout': 0.2,
        'batch_norm': 'before_act',
        'optimizer': {'type': 'adam', 'lr': 0.001},
        'batch_size': 64,
        'nb_epoch': 100,
    }
    # 探索するパラメータの空間を指定する
    param_space = {
        'input_dropout': hp.quniform('input_dropout', 0, 0.2, 0.05),
        'hidden_layers': hp.quniform('hidden_layers', 2, 4, 1),
        'hidden_units': hp.quniform('hidden_units', 32, 256, 32),
        'hidden_activation': hp.choice('hidden_activation', ['prelu', 'relu']),
        'hidden_dropout': hp.quniform('hidden_dropout', 0, 0.8, 0.05),
        'batch_norm': hp.choice('batch_norm', ['before_act', 'no']),
        'optimizer': hp.choice('optimizer',
                           [{'type': 'adam',
                             'lr': hp.loguniform('adam_lr', np.log(0.0001), np.log(0.01))},
                            {'type': 'sgd',
                             'lr': hp.loguniform('sgd_lr', np.log(0.00001), np.log(0.01))}]),
        'batch_size': hp.quniform('batch_size', 32, 256, 32)
    }

    #features = [
    #    "bow","bow_nva","bow_tf-idf","term_2-gram","term_3-gram","word2vec_mean","word2vec_pre_mean",
    #    "word2vec_fine-tuning", "doc2vec", "scdv", "bert"
    #]
    #features = [
    #    "tf-idf","n-gram","n-gram-tf-idf"
    #]
    #features = [
    #    "word2vec_mean", "word2vec_max", "word2vec_concat", "word2vec_hier", "fasttext_mean", "fasttext_max", "fasttext_concat", "fasttext_hier"
    #]
    features =  ["sdv"]

    NAME = "-".join(features)

    result = { }
    for i, name in enumerate(features):
        train_x = load_x_train(name)
        train_y = load_y_train(name)
        skf = StratifiedKFold(n_splits=6, shuffle=True, random_state=71)
        tr_idx, va_idx = list(skf.split(train_x, train_y))[0]
        tr_x, va_x = train_x[tr_idx], train_x[va_idx]
        tr_y, va_y = train_y[tr_idx], train_y[va_idx]

        # hyperoptによるパラメータ探索の実行
        max_evals = 100
        trials = Trials()
        history = []
        fmin(objective, param_space, algo=tpe.suggest, trials=trials, max_evals=max_evals)
        history = sorted(history, key=lambda tpl: tpl[1])
        best = history[0]
        logger.info(f'{name} - best params:{best[0]}, score:{best[1]:.4f}')
        
        for key, value in best[0].items():
            if key in result:
                result[key].append(value)
            else:
                result[key] = [value]
    res = pd.DataFrame.from_dict(
        result,
        orient='index',
        columns=features
    )
    res.to_csv(f'./model/tuning/MLP-{NAME}.csv')


This run of MLP-Twitter ran for 0:01:01 and logs are available locally at: /root/.hyperdash/logs/mlp-twitter/mlp-twitter_2019-12-16t05-47-07-346640.log

params: {'batch_norm': 'no', 'batch_size': 96.0, 'hidden_activation': 'prelu', 'hidden_dropout': 0.1, 'hidden_layers': 3.0, 'hidden_units': 32.0, 'input_dropout': 0.0, 'optimizer': {'lr': 0.00012954787338700032, 'type': 'sgd'}}, logloss: 1.3594
This run of MLP-Twitter ran for 0:00:14 and logs are available locally at: /root/.hyperdash/logs/mlp-twitter/mlp-twitter_2019-12-16t05-48-09-285218.log

params: {'batch_norm': 'no', 'batch_size': 96.0, 'hidden_activation': 'relu', 'hidden_dropout': 0.65, 'hidden_layers': 3.0, 'hidden_units': 128.0, 'input_dropout': 0.0, 'optimizer': {'lr': 0.0039860624868401215, 'type': 'adam'}}, logloss: 1.4350
This run of MLP-Twitter ran for 0:02:20 and logs are available locally at: /root/.hyperdash/logs/mlp-twitter/mlp-twitter_2019-12-16t05-48-23-852244.log

params: {'batch_norm': 'before_act', 'batch_size':

[2019-12-16 07:58:38] - sdv - best params:{'batch_norm': 'before_act', 'batch_size': 192.0, 'hidden_activation': 'relu', 'hidden_dropout': 0.25, 'hidden_layers': 3.0, 'hidden_units': 256.0, 'input_dropout': 0.2, 'optimizer': {'lr': 0.002885440473404958, 'type': 'sgd'}}, score:1.2507
